## Tarefa - Revisão de generalização
### Ajuste de características - Titanic Competition


In [8]:
# Desabilita warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

# Remome colunas indesejadas no final do tratamento dos dados
class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin', 'Age', 'Fare', 'Sex']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        if 'Name' not in self.colunasIndesejadas:
            X['Name'] = X['Name'].apply(extraiPronome)
            
            # Trata pronome de tratamento
            X['Name'] = X['Name'].replace('Mlle', 'Miss')
            X['Name'] = X['Name'].replace(['Ms','Mme'] , 'Mrs')
            X['Name'] = X['Name'].replace(['Lady', 'the Countess', 'Dona'] , 'Mrs')
            X['Name'] = X['Name'].replace(['Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Mr')
            # X['Name'] = X['Name'].replace(['Lady', 'the Countess', 'Dona', 
            #                                'Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Other')

        
        # Classe para o Sexo para drop 'Sex'
        X['Sex_Class'] = 0
        X.loc[ X['Sex'] == 'female', 'Sex_Class'] = 0
        X.loc[ X['Sex'] == 'male', 'Sex_Class'] = 1
        X['Sex_Class'] = X['Sex_Class'].astype(int)


        # Classes para as idades para drop 'Age'
        X['Age_Class'] = ''
        X['Age'].fillna(X['Age'].median(), inplace=True)
        
        X.loc[ X['Age'] < 16, 'Age_Class'] = 'children'
        X.loc[(X['Age'] >= 16) & (X['Age'] < 35), 'Age_Class'] = 'young'
        X.loc[(X['Age'] > 35) & (X['Age'] < 65), 'Age_Class'] = 'adult'
        X.loc[ X['Age'] >= 65, 'Age_Class'] = 'old'


        # Classes para tarifa para drop 'Fare'
        X['Fare_Class'] = ''
        X['Fare'].fillna(X['Fare'].median(), inplace=True)

        X.loc[ X['Fare'] < 20, 'Fare_Class'] = 'low_fare'
        X.loc[(X['Fare'] >= 20) & (X['Fare'] < 60), 'Fare_Class'] = 'median_fare'
        X.loc[(X['Fare'] >= 60) & (X['Fare'] < 120), 'Fare_Class'] = 'median_high_fare'
        X.loc[ X['Fare'] >= 120, 'Fare_Class'] = 'high_fare'


        # Cria característica FamilySize
        X['FamilySize'] = 0
        X['FamilySize'] = X['SibSp'] + X['Parch'] + 1
        X['FamilySize'] = X['FamilySize'].astype(int)
    
        # # Classifica pessoas viajando sozinhas e cria nova característica 'isAlone'
        # X['isAlone'] = 0
        # X.loc[(X['SibSp'] == 0) & (X['Parch'] == 0), 'isAlone'] = 1
        # X.loc[(X['SibSp'] > 0) | (X['Parch'] > 0), 'isAalone'] = 0
        # X['isAlone'] = X['isAlone'].astype(int)

        # Remove colunas indesejadas
        X = X.drop(self.colunasIndesejadas,axis=1)
        
        return X


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [False],
    'classificador__max_depth': [4],
    #'classificador__max_depth': [4, 5, 6, 7, 8],
    'classificador__criterion': ['gini', 'entropy'],
    'classificador__max_features': ['sqrt', 'log2'],
    #'classificador__max_features': [3, 4,'sqrt'],
    #'classificador__n_estimators' :[10, 20, 80, 100, 200],
    #'classificador__min_samples_split': [4, 5, 6],
    #'classificador__min_samples_leaf': [2, 3]
}

modelo = GridSearchCV(pipetotal, param_grid=parametros, n_jobs=-1)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

(array([0.81005587, 0.86516854, 0.81460674, 0.83146067, 0.85393258,
        0.81564246, 0.79213483, 0.85955056, 0.83146067, 0.85955056,
        0.84357542, 0.86516854, 0.78089888, 0.83146067, 0.85393258,
        0.82122905, 0.79213483, 0.80898876, 0.87078652, 0.87640449,
        0.84357542, 0.83707865, 0.83146067, 0.84831461, 0.81460674,
        0.8603352 , 0.78651685, 0.83707865, 0.85393258, 0.83707865,
        0.84916201, 0.84269663, 0.83707865, 0.81460674, 0.8258427 ,
        0.87709497, 0.83146067, 0.83707865, 0.78651685, 0.84269663,
        0.84357542, 0.82022472, 0.83707865, 0.85955056, 0.80337079,
        0.88268156, 0.84269663, 0.8258427 , 0.79213483, 0.83146067]),
 0.834219446362438,
 0.02484779691299112)

In [14]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)